### Coursera Assignment "Testing the Urban Convergence Hypothesis"

#### 1. Introduction

The hypothesis of urban convergence states that "urbanization creates similar landscapes regardless of the native biome in which cities are built" (https://heffernanlab.weebly.com/urban-homogenization.html), i.e. that cities become more alike over time. 

For the two cities that have been the subject of exercisies during the course, New York City (NYC) and Toronto, I wonder whether this hypothesis holds today, i.e. that neighborhoods in these two cities cannot be distinguished from each other by formal criteria. Specifically, I am interested whether it is possible to predict, using location data on the venues in a neighborhood, whether a given neighborhood is located in NYC or Toronto.

In line with The Urban Convergence Hypothesis, my prediction is thus that  it is not possible to successfully predict, with high accuracy, whether a given neighborhood, characterized by Foursquare data, is located in Toronto or NYC. 

The results obtained in this project will be relevant for both academics, that work on the urban convergence hypothesis, and for tourism agencied of NYC and Toronto.

#### 2. Data

To address this problem, I will use Foursquare location data on neighborhood venues that we have used in the previous exercises. For both cities, I'll use the following parameters for the Foursquare search:

- VERSION = '20180605' 
- LIMIT = 100 (the top 100 venues)
- Radius= 500 (500 meter radius)

I will use the neighborhood definitions used in the previous exercises. Data will be downloaded by executing Foursquare queries. 

#### 3. Methodology

The downloaded data will be split 70/30 into a training and a test dataset, and a Support Vector Machine (SVM) model will be trained. Using the test dataset, I'll predict whether a neighborhood is located in NYC or in Toronto and evaluate the accuracy of the model.

SVMs "scale relatively well to high dimensional data" (https://statinfer.com/204-6-8-svm-advantages-disadvantages-applications/), and will therefore be used in the present analysis of high-dimensional Foursquare data.

The *a-priori* set criterion for the acceptance or rejection of the hypothesis that the city cannot be predicted from Foursquare neighborhood data will be a Jaccard Similarity Score >0.7 (>0.7: hypothesis rejected, otherwise hypothesis accepted).

Steps in the analysis will be the following:

* 3.1 Import libraries  
* 3.2 Download and and format Tororonto neighborhoods dataset
* 3.3 Download and format NYC neighborhoods  
* 3.4 Set up parameters for Foursquare query 
* 3.5 Execute Foursquare query of venues in NYC  
* 3.6 Execute Foursquare query of venues in Toronto
* 3.7 Combine Toronto and NYC dataframes
* 3.8 Split data into training and test datasets
* 3.9 Train SVM model
* 3.10 Test SVM model
* 3.11 Calculate Jaccard index to evaluate the hypothesis

#### 3.1 Import libraries

In [2]:
import folium
import requests
import folium
import requests 
import json 

import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np 

from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### 3.2 Download and and format Tororonto neighborhoods dataset

Get the Toronto data from the wikipedia site by using the pd.read_html function, it is the first element that is returned

In [3]:
tor_web = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

tor_neighbor = tor_web[0]
tor_neighbor.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Rename the columns of the dataframe

In [4]:
tor_neighbor.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"},inplace=True)
tor_neighbor

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Drop cells that do not have an assigned borough.

In [5]:
tor_neighbor.drop(tor_neighbor.loc[tor_neighbor['Borough']=="Not assigned"].index, inplace=True)
tor_neighbor.head(100)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [6]:
tor_neighbor.shape

(211, 3)

Add geographical coordinates by reading the Geospatial_Coordinates.csv into a dataframe and merging this dataframe with the tor_neighbor dataframe

In [7]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)

geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two files on *PostalCode*

In [8]:
tor_neighbor_merge = pd.merge(tor_neighbor, geo, on='PostalCode')
tor_neighbor_merge.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


**3.3 Download and format NYC neighborhoods**

In [ ]:
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
#print('Data downloaded!')

Load data

In [25]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [26]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0:1]

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}}]

Transform to pandas dataframe

In [27]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [28]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [29]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [30]:
type(neighborhoods)

pandas.core.frame.DataFrame

#### 3.4 Set up Foursquare parameters

In [31]:
CLIENT_ID = 'RMLS3DLWXKID2GGBHCHWRTMPAFW0VZPE33PS3P00SJJKY3VB' # your Foursquare ID
CLIENT_SECRET = 'EUKE2PXBL2GZXVHNGWXCP5W1MSCFUW5OBEFWQUQR2YOTO4K0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = '100' 
Radius= '500' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('LIMIT:' + LIMIT)
print('Radius:' + Radius)

Your credentails:
CLIENT_ID: RMLS3DLWXKID2GGBHCHWRTMPAFW0VZPE33PS3P00SJJKY3VB
CLIENT_SECRET:EUKE2PXBL2GZXVHNGWXCP5W1MSCFUW5OBEFWQUQR2YOTO4K0
LIMIT:100
Radius:500


Define the function *get_nearby_venues* 

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### 3.5 Execute Foursquare query of venues in NYC

In [33]:
nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [41]:
nyc_venues.shape[0]

10408

Save to .csv

In [35]:
nyc_venues.to_csv("nyc_venues.csv")

Add column labeling the city

In [48]:
n = pd.Series(['nyc'])
n = n.repeat(nyc_venues.shape[0])
nyc_venues = nyc_venues.assign(city=pd.Series(n).values)

#### 3.6 Execute Foursquare query of venues in Toronto

In [36]:
tor_venues = getNearbyVenues(names=tor_neighbor_merge['Neighborhood'],
                                   latitudes=tor_neighbor_merge['Latitude'],
                                   longitudes=tor_neighbor_merge['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Not assigned
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

KeyError: 'groups'

In [37]:
tor_venues.shape

(4472, 7)

Save to .csv

In [38]:
tor_venues.to_csv("tor_venues.csv")

In [ ]:
Add column labeling the city

In [52]:
o = pd.Series(['tor'])
o = o.repeat(tor_venues.shape[0])

tor_venues = tor_venues.assign(city=pd.Series(o).values)

In [53]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,city
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,tor
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant,tor
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,tor
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,tor
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop,tor


#### 3.7 Combine Toronto and NYC dataframes

In [57]:
all = pd.concat([nyc_venues, tor_venues], ignore_index=True)
all.shape

(14880, 8)

Look at dataset

In [58]:
all.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,city
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,nyc
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy,nyc
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,nyc
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant,nyc
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop,nyc


Drop columns *Neighborhood*, *Neighborhood Latitude*, *Neighborhood Longitude*, *Venue Latitude*, *Venue Longitude*

In [62]:
all = all.drop(labels=['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude','Venue'],axis=1)

In [64]:
all.head(5)

,Venue Category,city
0,Dessert Shop,nyc
1,Pharmacy,nyc
2,Ice Cream Shop,nyc
3,Caribbean Restaurant,nyc
4,Donut Shop,nyc


Use one hot encoding technique to conver categorical varables to binary variables and append combine them into the *Feature* dataframe

In [65]:
Feature = pd.concat([all['city'],pd.get_dummies(all['Venue Category'])], axis=1)
Feature.head()

,city,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Sh

#### 3.8 Split data into training and test datasets

In [68]:
y = Feature['city'].values
y[0:5]

array(['nyc', 'nyc', 'nyc', 'nyc', 'nyc'], dtype=object)

In [72]:
X = Feature.drop(columns='city')

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (10416, 458) (10416,)
Test set: (4464, 458) (4464,)


#### 3.9 Train SVM Model

Here, I train the model using an radial basis function (RBF) kernel, which is commonly used with SVM.

In [87]:
from sklearn import svm
urbanSVM = svm.SVC(kernel='rbf')
urbanSVM.fit(X, y) 

/home/urs/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

**3.10 Test SVM model**

In [88]:
yhat_svm = urbanSVM.predict(X_test)
yhat_svm[0:5]

array(['nyc', 'nyc', 'nyc', 'nyc', 'nyc'], dtype=object)

**3.11 Calculate Jaccard index to evaluate the hypothesis**

In [89]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat_svm)

0.7007168458781362

#### 4. Results 

The Jaccard Similarity index is >0.7, therefore my hypothesis that the city cannot be predicted from Foursquare neighborhod data must be rejected. It appears that there are enough meaningful differences in neighborhood composition, evaluated using Foursqare data, between New York and Toronto. The cities thus have enough characteristic differences to discriminate them from each other using a SVM model. 

#### 5. Discussion

The analyses showed that the Urban Convergence Hypothesis does not hold when comparing NYC and Toronto, two metropolitan areas that are also similar in that they are the major financial centers of their respective countries.

However, this project compared only NYC and Toronto. To obtain more evidence for or against the Urban Convergence Hypothesis, many other cities should be compared with the methods described herein.

#### 6. Conclusion

The Urban Convergence Hypothesis did not to hold when examined in this project. Toronto and NYC both appear to have unique characteristics, a fact that may be exploited by the tourism agencies of both cities.